In [1]:
import numpy as np
import os
import glog as log
import copy

from tqdm import tqdm

import open3d as o3d
from open3d import JVisualizer
import pandas as pd

from evaluation.tools.mesh import Mesh
from evaluation.tools.mesh_evaluator import MeshEvaluator

# Rotation matrices:
# East North Up (ENU) frame to Unity's world frame of reference
enu_R_unity = np.array([[1, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]])
unity_R_enu = np.transpose(enu_R_unity)

# Right Handed frame to Unity's Left Handed frame of reference
righthand_R_lefthand = np.array([[1, 0, 0],
                                 [0, -1, 0],
                                 [0, 0, 1]])
lefthand_R_righthand = np.transpose(righthand_R_lefthand)

W1027 23:37:41.038026 21514 __init__.py:1410] /home/tonirv/Code/Kimera-VIO-Evaluation/evo-1/evo/tools/plot.py:33: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/tonirv/Code/Kimera-VIO-Evaluation/venv/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tonirv/Code/Kimera-VIO-Evaluation/venv/local/lib/python2.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/tonirv/Code/Kimera-VIO

In [2]:
# FILL PATHS BELOW
est_mesh_path = "/home/tonirv/Documents/uHumans2_VIO_vxblx/apartment_scene/uHumans2_apartment_s1_00h/mesh_DVIO.ply"
gt_mesh_path = "/home/tonirv/datasets/uHumans2/uHumans dataset V2.0 GT Meshes/apartment.ply"
visualize = False
semantic_labels_csv_path="/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/cfg/tesse_multiscene_archviz1_segmentation_mapping.csv"

mesh_eval = MeshEvaluator(est_mesh_path, gt_mesh_path, semantic_labels_csv_path)
mesh_eval.compare_meshes()


TypeError: __init__() takes exactly 3 arguments (4 given)

In [3]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)
print("Loading Estimated mesh...")
est_mesh_original = Mesh(est_mesh_path)

Loading Ground-truth mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 3614874 points and 1204958 triangles.
[[-3.492602  1.141617 -2.445021]
 [-3.492603  1.141617 -2.232179]
 [-3.663522  1.141617 -2.232179]
 ...
 [ 1.889433  4.119683 -4.40081 ]
 [ 1.888442  4.12226  -4.400809]
 [ 1.888441  4.120533 -4.400809]]
[[      0       1       2]
 [      3       4       5]
 [      6       7       8]
 ...
 [3614865 3614866 3614867]
 [3614868 3614869 3614870]
 [3614871 3614872 3614873]]

Loading Estimated mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 495836 points and 509972 triangles.
[[-3.175      0.025      2.72252  ]
 [-3.125      0.075      2.72331  ]
 [-3.125      0.025      2.72292  ]
 ...
 [-0.0393173 -2.175      1.575    ]
 [-0.0348831 -2.125      1.575    ]
 [-0.025     -2.099      1.575    ]]
[[     0      1      2]
 [     0      3      1]
 [     3      4      1]
 ...
 [473080 495832 473081]
 [473117 473118 495834]
 [495834 473118 495835]]



In [4]:
# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)
est_mesh = copy.deepcopy(est_mesh_original)

In [5]:
# Align Pointclouds Manually:
# est_mesh.mesh_o3d.translate([0, -5, 0])
# gt_mesh.transform_left(righthand_R_lefthand)
gt_mesh.transform_left(enu_R_unity)

Transforming mesh according to left matrix:
[[1 0 0]
 [0 0 1]
 [0 1 0]]


In [6]:
if visualize:
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.get_render_option().mesh_show_back_face = True
    vis.add_geometry(est_mesh.mesh_o3d)
    vis.add_geometry(gt_mesh.mesh_o3d)
    vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
    vis.run()
    vis.destroy_window()

In [7]:
NUMBER_OF_SAMPLES=1000000
gt_pcl = o3d.geometry.sample_points_uniformly(gt_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
# Don't sample estimated mesh, just pick vertices, otw you'll be mixing colors...
# est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
est_pcl = o3d.io.read_point_cloud(est_mesh_path)

In [8]:
# Calculate normals for nice visualization
# THIS COLORS THE PCL?????
#o3d.geometry.estimate_normals(
#        est_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))
#o3d.geometry.estimate_normals(
#        gt_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))

In [9]:
if visualize:
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.get_render_option().mesh_show_back_face = True
    vis.add_geometry(gt_pcl)
    vis.add_geometry(est_pcl)
    vis.run()
    vis.destroy_window()

In [10]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])

In [11]:
# ICP params
ICP_THRESHOLD = 1.5
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

In [12]:
# Visualize initial registration problem
if visualize:
    draw_registration_result(est_pcl, gt_pcl, trans_init)

In [13]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [14]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.141719, and correspondence_set size of 495836
Access transformation to get result.


In [15]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
    o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [16]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.131178, and correspondence_set size of 495836
Access transformation to get result.

Transformation is:
[[ 0.99990001  0.01407646 -0.00135215  0.1257669 ]
 [-0.01404803  0.99971831  0.01913011 -0.14393054]
 [ 0.00162105 -0.0191092   0.99981609 -0.01693472]
 [ 0.          0.          0.          1.        ]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 495836 elements.
Use numpy.asarray() to access data.



In [17]:
# Draw Registration Result
if visualize:
    draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)

In [18]:
# Draw Only Correspondences
if visualize:
    c2c_lines = o3d.geometry.create_line_set_from_point_cloud_correspondences(est_pcl, gt_pcl, correspondences)
    o3d.visualization.draw_geometries([c2c_lines])

In [19]:
# Draw PointClouds and Correspondences
if visualize:
    draw_correspondences(est_pcl, gt_pcl, c2c_lines)

In [20]:
# Import Semantic Labels
df = pd.read_csv(semantic_labels_csv_path)
#df

In [21]:
normalized_df = copy.deepcopy(df)
normalized_df['normalized_red'] = df['red'] / 255
normalized_df['normalized_green'] = df['green'] / 255
normalized_df['normalized_blue'] = df['blue'] / 255

normalized_df['normalized_red'] = normalized_df['normalized_red'].apply(lambda x: round(x, 5))
normalized_df['normalized_green'] = normalized_df['normalized_green'].apply(lambda x: round(x, 5))
normalized_df['normalized_blue'] = normalized_df['normalized_blue'].apply(lambda x: round(x, 5))

print(sum(np.isclose(normalized_df['normalized_red'], gt_pcl.colors[0][0])))

#normalized_df

0


In [22]:
# No need to hash, we have ids now
red_col = df['red'].to_numpy()
blue_col = df['blue'].to_numpy()
green_col = df['green'].to_numpy()

def f(x):    
    return str(x['normalized_red'])+str(x['normalized_green'])+str(x['normalized_blue'])

hashed_df = copy.deepcopy(normalized_df)
hashed_df['hash'] = hashed_df.apply(f, axis=1)
#hashed_df

In [23]:
# Generate table from color to id.
def label_from_color(color):
    # TODO(Toni): do you need to round again? isn't color already rounded?
    norm_r = round(color[0], 5)
    norm_g = round(color[1], 5)
    norm_b = round(color[2], 5)
    # TODO(Toni): can be greatly optimized... TOO SLOW NOW
    # TODO(Toni): you are comparing floats with == ......
    label_list = normalized_df.loc[(normalized_df['normalized_red'] == norm_r) & 
                  (normalized_df['normalized_green'] == norm_g) & 
                  (normalized_df['normalized_blue'] == norm_b)]['id'].unique().tolist()
    assert(len(label_list) > 0)
    return label_list[0]

In [24]:
def calc_corresp(est_pcl, gt_pcl, correspondences):
    total_negative_matches = 0
    total_positive_matches = 0
    total_correspondences = len(correspondences)
    
    # Compare labels between correspondences:
# Initialize dictionaries to 0:
    total_label_correspondences = {i:0 for i in normalized_df['id'].unique()}
    total_label_positive_matches = copy.deepcopy(total_label_correspondences)
    total_label_negative_matches = copy.deepcopy(total_label_correspondences)
    /la
    print("Total number of correspondences: ", total_correspondences)
    for correspondence in tqdm(correspondences):
        assert(len(correspondence) == 2)
        assert(correspondence[0] < len(est_pcl.colors))
        assert(correspondence[1] < len(gt_pcl.colors))
        est_label_id = label_from_color(est_pcl.colors[correspondence[0]])
        gt_label_id = label_from_color(gt_pcl.colors[correspondence[1]])
        if est_label_id == gt_label_id:
            total_positive_matches += 1
            total_label_positive_matches[est_label_id] += 1
        else: 
            total_negative_matches += 1
            total_label_negative_matches[est_label_id] += 1

    print("Positive matches: ", total_positive_matches)
    print("Negative matches: ", total_negative_matches)
    print("Total correspondences: ", total_correspondences)
    assert(total_correspondences == total_negative_matches + total_positive_matches)
    print ("Positive [%]: ", (total_positive_matches / total_correspondences * 100))
    print ("Negative [%]: ", (total_negative_matches / total_correspondences * 100))

In [25]:
calc_corresp(est_pcl, gt_pcl, correspondences)

('Total number of correspondences: ', 495836)


100%|██████████| 495836/495836 [22:40<00:00, 364.55it/s]

('Positive matches: ', 294371)
('Negative matches: ', 201465)
('Total correspondences: ', 495836)
('Positive [%]: ', 0)
('Negative [%]: ', 0)


In [26]:
def calc_per_label_corresp(est_pcl, gt_pcl, correspondences):
    for correspondence in tqdm(correspondences):
        est_pcl_color = est_pcl.colors[correspondence[0]]
        #color_hash = str(est_pcl_color[0])+str(est_pcl_color[1])+str(est_pcl_color[2])
        label_id = label_from_color(est_pcl_color)
        if np.allclose(est_pcl_color, gt_pcl.colors[correspondence[1]]):
            total_positive_matches[label_id] += 1
        else:
            total_negative_matches[label_id] += 1

In [27]:
calc_per_label_corresp(est_pcl, gt_pcl, correspondences)

  0%|          | 0/495836 [00:00<?, ?it/s]


NameError: global name 'total_negative_matches' is not defined

In [ ]:
for label_id in df['id'].unique().tolist():
    total_label_correspondences[label_id] = (total_positive_matches[label_id] + total_negative_matches[label_id])
print("Dict for total_label_correspondences: \n", total_label_correspondences)
positive_match = 0
negative_match = 0
correspondences = 0
for label_id in tqdm(df['id'].unique().tolist()):
    positive_match += total_label_positive_matches[label_id]
    negative_match += total_label_negative_matches[label_id]
    correspondences += total_label_correspondences[label_id]
    if (total_label_correspondences[label_id] > 0):
        print ("ID: {}".format(label_id))
        print ("Positive: {}  % ".format(total_positive_matches[label_id] / total_label_correspondences[label_id] * 100))
        print ("Negative: {}  % ".format(total_negative_matches[label_id] / total_label_correspondences[label_id] * 100))
print("Summary Positive label matches: {} % ".format(positive_match / correspondences * 100))
print("Summary Negative label matches: {} % ".format(negative_match / correspondences * 100))